In [1]:
from pathlib import Path
import pyarrow.dataset as ds    # type: ignore
import pyarrow.parquet as pq    # type: ignore
from rich import print as rprint
from rich.pretty import pprint

path = Path('tiny_raw_dataset')
path = Path('tiny_ingested_dataset')
# path = Path('tiny_raw_dataset')

single_file = sorted(path.glob('**/*.parquet'))[0]
single_file

PosixPath('tiny_ingested_dataset/year=2020/month=1/0c240110-2cde-44b9-80b3-2c02a190d843-0.parquet')

# Dataset read

This is only a pointer to the dataset:

In [2]:
dataset = ds.dataset(single_file, partitioning='hive')
rprint(dataset.schema)

datetime: timestamp[ns]
cat_col_00: dictionary<values=string, indices=int32, ordered=0>
cat_col_01: dictionary<values=string, indices=int32, ordered=0>
cat_col_02: dictionary<values=string, indices=int32, ordered=0>
num_col_00: double
num_col_01: double
num_col_02: double
num_col_03: double
num_col_04: double
num_col_05: double
num_col_06: double
num_col_07: double
num_col_08: double
num_col_09: double
batch: int64
day: int64
year: int32
month: int32

By default, a **fragment** corresponds to a parquet file,
but, for custom built datasets, it could map to file subset:

In [3]:
fragment = list(dataset.get_fragments())[0]
file = dataset.files[0]
assert fragment.path == file

rprint(f'{fragment.metadata=}')

fragment.metadata=<pyarrow._parquet.FileMetaData object at 0x107e8d300>
  created_by: parquet-cpp-arrow version 8.0.0
  num_columns: 16
  num_rows: 10
  num_row_groups: 1
  format_version: 2.6
  serialized_size: 3326

Number of row groups in the current file:

In [4]:
fragment.metadata.num_row_groups

1

In [5]:
row_group = fragment.row_groups[0]

rprint(f'* Row group metadata:\n{row_group.metadata}')
rprint('* Row group statistics:')
pprint(row_group.statistics)

* Row group metadata:
<pyarrow._parquet.RowGroupMetaData object at 0x12ec00a90>
  num_columns: 16
  num_rows: 10
  total_byte_size: 2320

* Row group statistics:

{
│   'datetime': {
│   │   'min': Timestamp('2020-01-10 00:00:00'),
│   │   'max': Timestamp('2020-01-10 21:36:00')
│   },
│   'cat_col_00': {'min': 'bar', 'max': 'foo'},
│   'cat_col_01': {'min': 'blue', 'max': 'red'},
│   'cat_col_02': {'min': 'buzz', 'max': 'fizz'},
│   'num_col_00': {'min': -2.110056045471277, 'max': 0.8939716512923349},
│   'num_col_01': {'min': -1.568944174498173, 'max': 1.879350389846965},
│   'num_col_02': {'min': -1.410558100187072, 'max': 1.5804565960208934},
│   'num_col_03': {'min': -1.1369827893393376, 'max': 1.7943357674660545},
│   'num_col_04': {'min': -1.7157342148490498, 'max': 0.7353560663789904},
│   'num_col_05': {'min': -0.9077183000237283, 'max': 1.516597270901757},
│   'num_col_06': {'min': -0.7914458562996477, 'max': 2.0548398612597034},
│   'num_col_07': {'min': -0.6976183740910009, 'max': 0.9037517913553875},
│   'num_col_08': {'min': -1.6386302265529098, 'max': 1.363856630019318},
│   'num_col_09': {'min': -0.9742933822826597, 'max': 1.7746551065417},
│   'batch': {'min': 9, 'max': 9},
│   'day': {'min': 10, 'max': 10}
}

# Pandas

In [6]:
import pandas as pd

In [7]:
pd.options.display.max_columns = 10

In [8]:
df = dataset.to_table().to_pandas()
df.head(3)

,datetime,cat_col_00,cat_col_01,cat_col_02,num_col_00,...,num_col_09,batch,day,year,month
0,2020-01-10 00:00:00,baz,red,fizz,-1.236827,...,-0.165189,9,10,2020,1
1,2020-01-10 02:24:00,foo,green,fizz,-0.989829,...,-0.974293,9,10,2020,1
2,2020-01-10 04:48:00,bar,blue,fizz,-0.857038,...,0.651265,9,10,2020,1



A pointer to a single parquet file

In [10]:
parquet_file = pq.ParquetFile(single_file)
rprint(f'* Parquet File Schema:\n{parquet_file.schema}\n')
rprint(f'* Parquet File Metadata:\n{parquet_file.metadata}\n')

* Parquet File Schema:
<pyarrow._parquet.ParquetSchema object at 0x12fa01e40>
required group field_id=-1 schema {
  optional int64 field_id=-1 datetime (Timestamp(isAdjustedToUTC=false, timeUnit=nanoseconds,
is_from_converted_type=false, force_set_converted_type=false));
  optional binary field_id=-1 cat_col_00 (String);
  optional binary field_id=-1 cat_col_01 (String);
  optional binary field_id=-1 cat_col_02 (String);
  optional double field_id=-1 num_col_00;
  optional double field_id=-1 num_col_01;
  optional double field_id=-1 num_col_02;
  optional double field_id=-1 num_col_03;
  optional double field_id=-1 num_col_04;
  optional double field_id=-1 num_col_05;
  optional double field_id=-1 num_col_06;
  optional double field_id=-1 num_col_07;
  optional double field_id=-1 num_col_08;
  optional double field_id=-1 num_col_09;
  optional int64 field_id=-1 batch;
  optional int64 field_id=-1 day;
}

* Parquet File Metadata:
<pyarrow._parquet.FileMetaData object at 0x12ec30400>
  created_by: parquet-cpp-arrow version 8.0.0
  num_columns: 16
  num_rows: 10
  num_row_groups: 1
  format_version: 2.6
  serialized_size: 3326